# Training a Machine Learning Model
Training machine learning models to predict FPL player points using engineered features.

## Models to train
- Random Forest, ensemble of decision trees 
- Linear Regression, baseline for comparison

## Features
- rolling_avg_points, player form last 5 games 
- opponent_difficulty, fixture difficulty (1 -10 scale)
- minutes, playing time a player gets (0-90 minutes)

## Steps
- Load features from 'fpl_features.csv',split the data into two parts 80% training and 20% testing, then test both models on the training data, assess on the test data & compare the performance.

## Performance Metrics 
- MAE (Mean Absolute Error), How far the predictions were off by 
- RMSE (Root Mean Squared Error), Prediction errors
- R^2 Score, How well the model predicts (0-1, higher score is better)

## Output
Random Forest normally achieves - 
MAE, 2.0-2.5 points
R^2 Score, 0.45-.055

In [33]:
# Importing libraries for ML training
# pandas for data manipulation, numpy for numerical operations
# sklearn for machine learning models and evaluation
# matplotlib for visualizations
# pickle to save the machine learning models
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import pickle

In [34]:
# Loading the features that are in the featureengineering.ipynb
# rolling averages, opponent difficulty and minutes
# these will be used to train the model

# reading the csv file that has all the features
df = pd.read_csv('fpl_features.csv')

# Showing how many records, players & gameweeks were loaded
print(f"{len(df)} records")
print(f"{df['player_id'].nunique()} players")
print(f"{df['gameweek'].min()} to {df['gameweek'].max()} GW")

# showing the data loaded properly 
print(df[['name', 'gameweek', 'rolling_avg_points', 'opponent_difficulty', 
          'minutes', 'total_points']].head())

10329 records
756 players
1 to 14 GW
   name  gameweek  rolling_avg_points  opponent_difficulty  minutes  \
0  Raya         1                 0.0                  6.9       90   
1  Raya         2                 0.0                  6.5       90   
2  Raya         3                 0.0                  4.9       90   
3  Raya         4                 0.0                  7.9       90   
4  Raya         5                 0.0                  1.2       90   

   total_points  
0            10  
1             6  
2             2  
3             6  
4             2  


In [35]:
# preparing data for machine learning
# splitting it into features x and y
# removing players who didnt play

# features x and y
# x - input features (what the moodel learns from)
# y - target variable (what the model iis trying to predict)
# putting the features into x for the model to learn from
features = ['rolling_avg_points', 'opponent_difficulty', 'minutes']
x = df[features]
y = df['total_points']

# showing the features we are using for the model 
# and the total points
print(f"features (x) {features}")
print(f"target (y) total points")

# splitting data into two parts training 80% and testing 20%
# training set, model learns patterns from this data
# testing set, model evaluates on this to test generalization
# making the random_state - 42, makes the split reproducible
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42
)

print(f"\ntraining set: {len(x_train)} records (80%)")
print(f"test set: {len(x_test)} records (20%)")

features (x) ['rolling_avg_points', 'opponent_difficulty', 'minutes']
target (y) total points

training set: 8263 records (80%)
test set: 2066 records (20%)


In [36]:
# implementing random forest an ensemble learning method
# makes multiple decision trees & averages their predictions
# it is good at capturing non-linear relationships
# using 100 n_estimators, which is 100 decision trees
# random_state=42 for reproducability
# n_jobs=1 uses all cpu cores for faster training
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
# fit() trains the model on the training data
rf_model.fit(x_train, y_train)

# predictions on test data
# showing how well the model works on unseen data
rf_predictions = rf_model.predict(x_test)

# showing model performance using standard metrics
# MAE (mean absolute error) average prediction error in points
# the lower the better, shows the average points it is off by
rf_mae = mean_absolute_error(y_test, rf_predictions)
print(f"MAE (mean absolute error):  {rf_mae:.2f} points")
print(f"On average, predictions are off by {rf_mae:.2f} points")

# RMSE (root mean squared error) penalizes large errors more than mae
# the lower the better
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_predictions))
print(f"RMSE (root mean squared error): {rf_rmse:.2f} points")

# R^2 Score, how well the model predicts
# 1.0 means perfect predictions, 0.0 means the model is the same as just getting the average
# the higher the bettter
rf_r2 = r2_score(y_test, rf_predictions)
print(f"R² Score: {rf_r2:.3f}")
print(f"Model shows how well the model predicts points {rf_r2*100:.1f}%")

# Finding the R² score
if rf_r2 > 0.5:
    print(f"Strong performance, Model explains {rf_r2*100:.1f}% of variance")
elif rf_r2 > 0.3:
    print(f"Moderate performance, Model explains {rf_r2*100:.1f}% of variance")
elif rf_r2 > 0:
    print(f"Weak performance, Model explains {rf_r2*100:.1f}% of variance")
else:
    print(f"Poor performance, Model didn't learn (negative R²)")


# showing the predictions to see how close they are 
print("Sample Predictions (Random Forest):")
comparison = pd.DataFrame({
    'Actual': y_test[:10].values,
    'Predicted': rf_predictions[:10].round(1),
    'Error': abs(y_test[:10].values - rf_predictions[:10]).round(1)
})
print(comparison)

MAE (mean absolute error):  0.83 points
On average, predictions are off by 0.83 points
RMSE (root mean squared error): 2.01 points
R² Score: 0.315
Model shows how well the model predicts points 31.5%
Moderate performance, Model explains 31.5% of variance
Sample Predictions (Random Forest):
   Actual  Predicted  Error
0       0        0.0    0.0
1       0        0.0    0.0
2       1        1.3    0.3
3       1        7.1    6.1
4       1        1.5    0.5
5       2        1.7    0.3
6       0        0.0    0.0
7       0        0.0    0.0
8       0        0.0    0.0
9       0        0.0    0.0


In [37]:
# Making a baseline (Linear Regression) to compare ml models to
# This model assumes a straight line relationship between 
# inputs and outputs

# training linear regression model
# no features needed
lr_model = LinearRegression()
# fit() finds the best linear equation to fit the data
lr_model.fit(x_train, y_train)

# showing how well the model works
lr_predictions = lr_model.predict(x_test)

# showing model performance using standard regression metrics
# MAE (mean absolute error) average prediction error in points
# the lower the better, shows the average points it is off by
lr_mae = mean_absolute_error(y_test, lr_predictions)
print(f"MAE (mean absolute error):  {lr_mae:.2f} points")

# RMSE (root mean squared error) penalizes large errors more than mae
# the lower the better
lr_rmse = np.sqrt(mean_squared_error(y_test, lr_predictions))
print(f"RMSE (root mean squared error): {lr_rmse:.2f} points")

# R^2 Score, how well the model predicts
# 1.0 means perfect predictions, 0.0 means the model is the same as just getting the average
# the higher the bettter
lr_r2 = r2_score(y_test, lr_predictions)
print(f"R² Score: {lr_r2:.3f}")
print(f"Model shows how well the model predicts points {rf_r2*100:.1f}%")

# showing the predictions to see how close they are 
print("\nExample Predictions (Linear Regression):")
print("-"*60)
comparison = pd.DataFrame({
    'Actual': y_test[:10].values,
    'Predicted': lr_predictions[:10].round(1)
})
print(comparison)

MAE (mean absolute error):  0.86 points
RMSE (root mean squared error): 1.78 points
R² Score: 0.462
Model shows how well the model predicts points 31.5%

Example Predictions (Linear Regression):
------------------------------------------------------------
   Actual  Predicted
0       0        0.1
1       0        0.0
2       1        1.4
3       1        4.2
4       1        0.7
5       2        3.6
6       0        0.0
7       0        0.1
8       0        0.2
9       0       -0.1


In [38]:
# Comparing both models to see the best one
# The model that has the lowest MAE score is the best
# in fpl the average error matters more than then perfect predicition

# creating a comparison dataframe/table with all the metrics
comparison_df = pd.DataFrame({
    'Model': ['Random Forest', 'Linear Regression'],
    'MAE': [rf_mae, lr_mae],
    'RMSE': [rf_rmse, lr_rmse],
    'R² Score': [rf_r2, lr_r2]
})

# Printing out the comparison dataframe/table
print("\nComparing Random Forest & Linear Regression")
print(comparison_df.to_string(index=False))

# Seeing what the best model is, the lowest MAE score
# The lower the MAE , the smaller average prediction error
bmn = comparison_df.loc[comparison_df['MAE'].idxmin(), 'Model']
print(f"\nBest Model (Lowest MAE): {bmn}")

# This shows what feature matters the most
print("\nFeature Importance (Random Forest):")
# Geting the performance score of each feature
# The higher the score th more important they are
feature_importance = pd.DataFrame({
    'Feature': features,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

print(feature_importance.to_string(index=False))

# explaining what the importance of the features
print("\nExplanation:")
# for the features in the table show what % of performance each has
for idx, row in feature_importance.iterrows():
    print(f"{row['Feature']}: {row['Importance']*100:.1f}% importance")



Comparing Random Forest & Linear Regression
            Model      MAE     RMSE  R² Score
    Random Forest 0.825537 2.006421  0.315409
Linear Regression 0.859606 1.779066  0.461767

Best Model (Lowest MAE): Random Forest

Feature Importance (Random Forest):
            Feature  Importance
            minutes    0.703916
 rolling_avg_points    0.164969
opponent_difficulty    0.131115

Explanation:
minutes: 70.4% importance
rolling_avg_points: 16.5% importance
opponent_difficulty: 13.1% importance


In [39]:
# Testing random forest on actual players
# Getting the latest gameweek data for realistic testing
latest_gw = df['gameweek'].max()
print(f"\nUsing gameweek {latest_gw} data for predictions")

# getting the top 10 players by their form
players = df[df['gameweek'] == latest_gw].nlargest(10, 'rolling_avg_points')

# adding the features to the players 
x_players = players[features]
# generating precitions using random forest
predictions_rf = rf_model.predict(x_players)

# making a predictions dataframe/table to show to actual points v the predicted points  
predictions = pd.DataFrame({
    'Player': players['name'].values,
    'Form': players['rolling_avg_points'].values,
    'Opponent_Diff': players['opponent_difficulty'].values,
    'Minutes': players['minutes'].values,
    'Actual_Points': players['total_points'].values,
    'Predicted_Points': predictions_rf.round(1)
})

# printing out the predictions table of actual points v predicted point
print("\nTop 10 players, Actual v Predicted")
print(predictions.to_string(index=False))

# calculating thhe average error of these predictions
# showing how accurate random forest is on the players
predictions['Error'] = abs(predictions['Actual_Points'] - predictions['Predicted_Points'])
print(f"\nAverage prediction error. {predictions['Error'].mean():.2f} points")


Using gameweek 14 data for predictions

Top 10 players, Actual v Predicted
       Player  Form  Opponent_Diff  Minutes  Actual_Points  Predicted_Points
          Eze   7.6            6.3        0              0               0.0
       Thiago   7.4            1.0        0              0               0.0
     Casemiro   7.2            7.8        0              0               0.0
      O.Dango   6.8            1.0        0              0               0.0
     Martinez   6.6            7.4        0              0               0.0
Dewsbury-Hall   6.6            5.4       90              3               3.1
         Rice   6.4            6.3        0              0               0.0
      Lacroix   6.4            8.0        0              0               0.0
     Bruno G.   6.4            4.6       45              7               5.0
       Minteh   6.2            6.6        0              0               0.0

Average prediction error. 0.21 points


In [40]:
# Saving Random Forest using pickle
# saving to pickle so can the load the file later to make predictions without retraining
model_filename = 'fpl_predictor_model.pkl'
# wb to write the file in binary mode
with open(model_filename, 'wb') as file:
    pickle.dump(rf_model, file)

print(f"Random Forest saved to {model_filename}")

# creating a summary of the ml training
summary = f"""
Random Forest
Training Date - {pd.Timestamp.now()}

Features Used -
rolling_avg_points - Player form from the last 5 games
opponent_difficulty - Fixture difficulty from a scale of 1-10
minutes - How many minutes a player gets from 0 -90 minutes

Performance Metrics -
MAE - {rf_mae:.2f} points
RMSE - {rf_rmse:.2f} points
R² Score - {rf_r2:.3f}

Training Data -
- Records: {len(x_train)}
- Players: {df['player_id'].nunique()}
- Gameweeks: {df['gameweek'].min()} to {df['gameweek'].max()}
"""

# saving the summary to a txt file
with open('model_summary.txt', 'w') as f:
    f.write(summary)

print("Summary of Random Forest saved to model_summary.txt")


Random Forest saved to fpl_predictor_model.pkl
Summary of Random Forest saved to model_summary.txt
